# 📊 EDA Template - 数据探索与预处理标准流程

每次新项目/比赛时复制此模板，按需修改使用。

**使用方法：**
1. 修改 Section 1 的文件路径
2. 修改 Section 5 的 `TARGET` 变量名
3. 逐个 Cell 运行，按需调整

---
## 0. 导入常用库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# 显示设置
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
plt.style.use('seaborn-v0_8-whitegrid')

import warnings
warnings.filterwarnings('ignore')

print('✅ Libraries loaded')

---
## 1. 加载数据

⚠️ **TODO: 修改文件路径**

In [ ]:
# TODO: 修改文件路径
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

---
## 2. 快速概览

In [ ]:
def quick_overview(df, name="DataFrame"):
    """数据集快速概览"""
    print(f"\n{'='*60}")
    print(f"📊 {name} Overview")
    print(f"{'='*60}")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"\n--- Data Types ---")
    print(df.dtypes.value_counts())
    return df.head()

In [ ]:
quick_overview(train, "Train")

In [ ]:
quick_overview(test, "Test")

In [ ]:
# 详细信息
train.info()

In [ ]:
# 数值统计
train.describe()

In [ ]:
# 类别统计
train.describe(include='object')

---
## 3. 缺失值分析

In [ ]:
def missing_analysis(df, name="DataFrame"):
    """缺失值分析"""
    missing = df.isnull().sum()
    missing_pct = df.isnull().mean() * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    }).sort_values('Missing %', ascending=False)
    
    missing_df = missing_df[missing_df['Missing Count'] > 0]
    
    print(f"\n{'='*60}")
    print(f"🔍 {name} Missing Values")
    print(f"{'='*60}")
    
    if len(missing_df) == 0:
        print("✅ No missing values!")
    else:
        print(f"Columns with missing values: {len(missing_df)}")
    
    return missing_df

In [ ]:
train_missing = missing_analysis(train, "Train")
train_missing

In [ ]:
test_missing = missing_analysis(test, "Test")
test_missing

---
## 4. 重复值检查

In [ ]:
def duplicate_check(df, name="DataFrame"):
    """重复值检查"""
    dup_count = df.duplicated().sum()
    print(f"🔄 {name} Duplicates: {dup_count} ({dup_count/len(df)*100:.2f}%)")
    return dup_count

In [ ]:
train_dups = duplicate_check(train, "Train")
test_dups = duplicate_check(test, "Test")

In [ ]:
# 如需删除重复值，取消下行注释
# train = train.drop_duplicates()

---
## 5. 目标变量分析

⚠️ **TODO: 修改目标变量名**

In [ ]:
# TODO: 修改目标变量名
TARGET = 'target'

In [ ]:
# 值分布
print("--- Value Counts ---")
print(train[TARGET].value_counts())
print("\n--- Value Proportions ---")
print(train[TARGET].value_counts(normalize=True))

In [ ]:
# 目标变量可视化
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 柱状图
train[TARGET].value_counts().plot(kind='bar', ax=axes[0], edgecolor='black')
axes[0].set_title(f'{TARGET} Distribution')
axes[0].set_ylabel('Count')

# 饼图
train[TARGET].value_counts().plot(kind='pie', autopct='%1.1f%%', ax=axes[1])
axes[1].set_title(f'{TARGET} Proportion')

plt.tight_layout()
plt.show()

---
## 6. 特征类型分离

In [ ]:
def separate_features(df, target_col=None):
    """分离数值特征和类别特征"""
    cols = [c for c in df.columns if c != target_col]
    
    num_cols = df[cols].select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = df[cols].select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"Numeric features: {len(num_cols)}")
    print(f"Categorical features: {len(cat_cols)}")
    
    return num_cols, cat_cols

In [ ]:
num_cols, cat_cols = separate_features(train, TARGET)

print(f"\nNumeric: {num_cols}")
print(f"\nCategorical: {cat_cols}")

---
## 7. 数值特征分析

In [ ]:
def numeric_distribution(df, num_cols, n_cols=4):
    """数值特征分布可视化"""
    if len(num_cols) == 0:
        print("No numeric features to analyze.")
        return
    
    n_features = min(len(num_cols), 16)  # 最多显示16个
    n_rows = (n_features - 1) // n_cols + 1
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 3*n_rows))
    axes = axes.flatten()
    
    for i, col in enumerate(num_cols[:n_features]):
        df[col].hist(ax=axes[i], bins=30, edgecolor='black')
        axes[i].set_title(col)
    
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
numeric_distribution(train, num_cols)

---
## 8. 类别特征分析

In [ ]:
def categorical_summary(df, cat_cols):
    """类别特征唯一值统计"""
    if len(cat_cols) == 0:
        print("No categorical features to analyze.")
        return None
    
    cat_info = pd.DataFrame({
        'Unique Values': [df[col].nunique() for col in cat_cols],
        'Sample Values': [df[col].dropna().unique()[:3].tolist() for col in cat_cols]
    }, index=cat_cols).sort_values('Unique Values', ascending=False)
    
    return cat_info

In [ ]:
cat_info = categorical_summary(train, cat_cols)
cat_info

---
## 9. 相关性分析

In [ ]:
# 相关性矩阵热力图
if len(num_cols) > 1:
    plt.figure(figsize=(12, 10))
    corr_matrix = train[num_cols].corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
    plt.title('Correlation Matrix')
    plt.tight_layout()
    plt.show()

In [ ]:
# 与目标变量的相关性 (仅当目标是数值型时)
if TARGET in train.columns and train[TARGET].dtype in [np.number, 'int64', 'float64']:
    correlations = train[num_cols + [TARGET]].corr()[TARGET].drop(TARGET)
    correlations = correlations.sort_values(ascending=False)
    
    print(f"Correlation with {TARGET}:")
    print(correlations)
    
    # 可视化
    plt.figure(figsize=(10, max(6, len(correlations) * 0.3)))
    correlations.plot(kind='barh')
    plt.title(f'Feature Correlation with {TARGET}')
    plt.xlabel('Correlation')
    plt.tight_layout()
    plt.show()

---
## 10. 数据划分

In [ ]:
X = train.drop(TARGET, axis=1)
y = train[TARGET]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # 分类任务使用，回归任务删除此参数
)

print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"\ny_train distribution:")
print(y_train.value_counts(normalize=True))
print(f"\ny_val distribution:")
print(y_val.value_counts(normalize=True))

---
## 11. 常用预处理函数

In [ ]:
def fill_missing(df, num_strategy='median', cat_strategy='mode'):
    """填充缺失值
    
    Args:
        df: DataFrame
        num_strategy: 'median' | 'mean' | 'zero'
        cat_strategy: 'mode' | 'unknown'
    """
    df = df.copy()
    
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype in ['int64', 'float64']:
                if num_strategy == 'median':
                    df[col].fillna(df[col].median(), inplace=True)
                elif num_strategy == 'mean':
                    df[col].fillna(df[col].mean(), inplace=True)
                elif num_strategy == 'zero':
                    df[col].fillna(0, inplace=True)
            else:
                if cat_strategy == 'mode':
                    df[col].fillna(df[col].mode()[0], inplace=True)
                elif cat_strategy == 'unknown':
                    df[col].fillna('Unknown', inplace=True)
    
    return df

In [ ]:
def encode_categorical(df, cat_cols, encoding='label'):
    """编码类别特征
    
    Args:
        df: DataFrame
        cat_cols: 类别列名列表
        encoding: 'label' | 'onehot'
    """
    df = df.copy()
    
    if encoding == 'label':
        from sklearn.preprocessing import LabelEncoder
        for col in cat_cols:
            if col in df.columns:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col].astype(str))
    
    elif encoding == 'onehot':
        df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    
    return df

In [ ]:
# 使用示例（按需取消注释）
# train_processed = fill_missing(train)
# train_processed = encode_categorical(train_processed, cat_cols, encoding='label')

---
## 12. 导出数据（可选）

In [ ]:
# 导出处理后的数据
# train_processed.to_csv('train_processed.csv', index=False)
# test_processed.to_csv('test_processed.csv', index=False)
# print('✅ Data exported')

---
## ✅ EDA Complete!

**下一步：**
- [ ] 特征工程
- [ ] 建立 Baseline 模型
- [ ] 交叉验证
- [ ] 模型调优